# Coursera Capstone
IBM Applied Data Science Capstone
Opening a Seafood restaurant in Tampa (Florida)
By: V.K
June 2020

This analysis is part of my Applied Data Science Capstone course in Coursera. In this assignment I use my new skills in exploring, segmenting and clustering data and for the first time I prepare full report on my findings. Instruction for this assignment was to compare neighborhoods or cities and use Foursquare location data to solve a business problem.
In my analysis I explore what would be the best place to establish opening a restaurant in Tampa.


Table of Contents:  
1.	Business problem
2.	Data Description
3.	Methodology
4.	Results
5.	Conclusion

## 1 Business problem

Tampa is the 48th most-populous city in the U.S. and the third-largest city in Florida after Miami and Jacksonville. The population of the Tampa Bay MSA is estimated at 3,142,663 people as of 2018.
A recently released report analyzed 236 cities across the U.S., including the 150 most populous, to determine the 50 best cities to open a restaurant.
Bid on Equipment named Tampa the No. 13 best city to open a restaurant, using the key factors of annual restaurant sales per capita, competition and market saturation (restaurants per capita), workforce (the number of restaurant industry workers per capita) and medium income in each city
Let suppose, an investor has enough time and moneys, as well as a passion to open the best eating spot in Tampa. What type of restaurant would it be? What would be the best location for it?
Target audience: investors, entrepreneurs, and chefs interest in opening a restaurant in Tampa, who may require objective advice regarding what type exactly it should be opened.

## 2 Data

To solve the problem, we will need the following data:

1. US ZIP Code data. https://simplemaps.com/data/us-zips 
2. Postal Codes data.https://www.geonames.org/postal-codes/US/FL/florida.html
3. Tampa City Data. https://www.tampagov.net/
4. Foursquare location data. https://foursquare.com/


## Import data

In [1]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
import json # library to handle JSON files

# import k-means from clustering stage
from sklearn.cluster import KMeans

# For web scraping
import urllib.request # Open URLs

print('Libraries imported.')

Libraries imported.


In [2]:
conda install -c conda-forge folium

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.8.3
  latest version: 4.8.4

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - folium


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    branca-0.4.1               |             py_0          26 KB  conda-forge
    brotlipy-0.7.0             |py36h8c4c3a4_1000         346 KB  conda-forge
    chardet-3.0.4              |py36h9f0ad1d_1006         188 KB  conda-forge
    cryptography-3.1           |   py36h45558ae_0         615 KB  conda-forge
    folium-0.11.0              |             py_0          61 KB  conda-forge
    openssl-1.1.1g             |       h516909a_1         2.1 MB  conda-forge
    pysocks-1.7.1              |   py36h9f0ad1d_1          2

In [3]:
import folium # map rendering library

In [4]:
conda install -c conda-forge geopy

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.8.3
  latest version: 4.8.4

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-2.0.0                |     pyh9f0ad1d_0          63 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          97 KB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.50-py_0
  geopy              conda-forge/noarch::geopy-2.0.0-pyh9f0ad1d_0



geopy-2.0.0          | 63 KB     | ##########################

In [5]:
from geopy.geocoders import Nominatim

In [6]:
import pandas as pd
import numpy as np

## Load data

In [7]:
# Get the data
df1 = pd.read_csv('tnd.csv')
df1.head()

,PostalCode,Country,State,County,Neighborhood,Unnamed: 5
0,33602,United States,Florida,Hillsborough,Harbour Island,NaN
1,33603,United States,Florida,Hillsborough,Lowry Park Central,NaN
2,33604,United States,Florida,Hillsborough,River Grove,NaN
3,33605,United States,Florida,Hillsborough,East Tampa,NaN
4,33606,United States,Florida,Hillsborough,Historic Hyde Park North,NaN


In [8]:
# Let's take the columns we need
df1 = df1[['PostalCode','County','Neighborhood']].reset_index(drop=True)
df1.head()

,PostalCode,County,Neighborhood
0,33602,Hillsborough,Harbour Island
1,33603,Hillsborough,Lowry Park Central
2,33604,Hillsborough,River Grove
3,33605,Hillsborough,East Tampa
4,33606,Hillsborough,Historic Hyde Park North


In [9]:
# Get the data
df_tampazipcode = pd.read_csv('tampazipcode.csv')
df_tampazipcode.head()

,Code,Country,Admin1,Admin2,Admin3,Admin4,Population,Households per ZIP Code,Average House Value,Average Income Per Household,Male Population,Female Population,Male Median Age,Female Median Age,Unemployment Rate,Unnamed: 15
0,33602,United States,Florida,Hillsborough,27.961,-82.460,17412,6004,272000,67945,6092,5423,35.8,34.5,12.1%,NaN
1,33603,United States,Florida,Hillsborough,27.985,-82.463,20303,7721,147600,40078,9421,9679,35.4,37.5,12.1%,NaN
2,33604,United States,Florida,Hillsborough,28.017,-82.458,38971,13944,118400,37316,17443,18042,35.6,37.0,12.1%,NaN
3,33605,United States,Florida,Hillsborough,27.967,-82.433,18689,6481,84500,26121,8469,8604,34.6,35.8,12.1%,NaN
4,33606,United States,Florida,Hillsborough,27.948,-82.458,18271,7517,452900,74855,8632,9114,32.1,29.4,12.1%,NaN


In [10]:
# Rename the columns
df_tampazipcode.rename(columns = {'Code': 'PostalCode', 'Admin1':'State', 'Admin2':'County', 'Admin3':'Latitude', 'Admin4':'Longitude'}, inplace=True)
df_tampazipcode.head()

,PostalCode,Country,State,County,Latitude,Longitude,Population,Households per ZIP Code,Average House Value,Average Income Per Household,Male Population,Female Population,Male Median Age,Female Median Age,Unemployment Rate,Unnamed: 15
0,33602,United States,Florida,Hillsborough,27.961,-82.460,17412,6004,272000,67945,6092,5423,35.8,34.5,12.1%,NaN
1,33603,United States,Florida,Hillsborough,27.985,-82.463,20303,7721,147600,40078,9421,9679,35.4,37.5,12.1%,NaN
2,33604,United States,Florida,Hillsborough,28.017,-82.458,38971,13944,118400,37316,17443,18042,35.6,37.0,12.1%,NaN
3,33605,United States,Florida,Hillsborough,27.967,-82.433,18689,6481,84500,26121,8469,8604,34.6,35.8,12.1%,NaN
4,33606,United States,Florida,Hillsborough,27.948,-82.458,18271,7517,452900,74855,8632,9114,32.1,29.4,12.1%,NaN


In [11]:
# Let's take the columns we need
df_tampazipcode = df_tampazipcode[['PostalCode','Latitude','Longitude']].reset_index(drop=True)
df_tampazipcode.head()

,PostalCode,Latitude,Longitude
0,33602,27.961,-82.460
1,33603,27.985,-82.463
2,33604,28.017,-82.458
3,33605,27.967,-82.433
4,33606,27.948,-82.458


In [12]:
df_tampazipcode.shape

(24, 3)

In [13]:
# Merge the dataframes
tampa_data = pd.merge(df1, df_tampazipcode, on='PostalCode', how='inner')
tampa_data

,PostalCode,County,Neighborhood,Latitude,Longitude
0,33602,Hillsborough,Harbour Island,27.961,-82.460
1,33603,Hillsborough,Lowry Park Central,27.985,-82.463
2,33604,Hillsborough,River Grove,28.017,-82.458
3,33605,Hillsborough,East Tampa,27.967,-82.433
4,33606,Hillsborough,Historic Hyde Park North,27.948,-82.458
5,33607,Hillsborough,Carver City-Lincoln Gardens,27.963,-82.490
6,33609,Hillsborough,Audubon Park,27.942,-82.506
7,33610,Hillsborough,Eastern Heights,27.995,-82.405
8,33611,Hillsborough,Ballast Point,27.891,-82.507
9,33612,Hillsborough,Forest Hills,28.050,-82.450


In [14]:
tampa_data.shape

(24, 5)

In [15]:
# Get the latitude and longitude values of Tampa

address = 'Tampa, FL'

geolocator = Nominatim(user_agent="toro_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Tampa are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Tampa are 27.9477595, -82.458444.


In [16]:
# create map of Tampa using latitude and longitude values
map_tampa = folium.Map(location=[27.9477595, -82.458444], zoom_start=11)
map_tampa

In [17]:
# add markers to map
for lat, lng, Neighborhood in zip(tampa_data['Latitude'], tampa_data['Longitude'], tampa_data['Neighborhood']):
    label = folium.Popup(Neighborhood, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup='PostalCode',
        color='red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_tampa)  
    
map_tampa

In [18]:
map_tampa.save('mymap.html')

# Data Foursquare

In [19]:
# Import my passwords for Foursquare
import os
import sys
CLIENT_ID = 'UROHXRU5BEXZY1ZRHH3E2C0OOSZ4O3423DTR5KQDLVADSTNI' # your Foursquare ID
CLIENT_SECRET = 'JLTXGBSH4EOISXQXMFOPVQP3HAQJ4T23HNRJAHV2WLSUZADR' # your Foursquare Secret
VERSION = '20200629' # Foursquare API version

# defining radius and limit of venues to get
radius=500
LIMIT=100

In [20]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [21]:
tampa_venues = getNearbyVenues(names=tampa_data['Neighborhood'],
                                   latitudes=tampa_data['Latitude'],
                                   longitudes=tampa_data['Longitude']
                                  )

Harbour Island
Lowry Park Central
River Grove
East Tampa
Historic Hyde Park North
Carver City-Lincoln Gardens
Audubon Park
Eastern Heights
Ballast Point
Forest Hills
Tampa Palms
Drew Park
College Hill
Gandy-Sun Bay South
Greater Carrollwood
Dixie Farms
Greater Northdale
Citrus Park
Westchester of Hillsborough
Bayshore Beautiful
Northwest Park
Forest at Countryway
Tampa Palms
Hunters Green


In [48]:
tampa_venues.groupby('Venue Category').count()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude
Venue Category,,,,,,
American Restaurant,6,6,6,6,6,6
Argentinian Restaurant,1,1,1,1,1,1
Art Gallery,1,1,1,1,1,1
Asian Restaurant,1,1,1,1,1,1
Athletics & Sports,1,1,1,1,1,1
BBQ Joint,2,2,2,2,2,2
Bagel Shop,2,2,2,2,2,2
Bakery,1,1,1,1,1,1
Bar,5,5,5,5,5,5


## 3. Analyze Each Neighborhood

In [49]:
# Let's check the size of the resulting dataframe

print(tampa_venues.shape)
tampa_venues.head()

(219, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Harbour Island,27.961,-82.46,Hidden Springs Ale Works,27.959310,-82.460215,Brewery
1,Harbour Island,27.961,-82.46,YMCA,27.962331,-82.459746,Gym / Fitness Center
2,Harbour Island,27.961,-82.46,Ulele,27.959821,-82.462889,New American Restaurant
3,Harbour Island,27.961,-82.46,Foundation Coffee,27.958667,-82.460011,Coffee Shop
4,Harbour Island,27.961,-82.46,Water Works Park,27.959167,-82.462802,Park


In [53]:
# Check how many venues were returned for each neighborhood
tampa_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Audubon Park,40,40,40,40,40,40
Ballast Point,25,25,25,25,25,25
Bayshore Beautiful,6,6,6,6,6,6
Carver City-Lincoln Gardens,10,10,10,10,10,10
Citrus Park,5,5,5,5,5,5
College Hill,4,4,4,4,4,4
Dixie Farms,5,5,5,5,5,5
Drew Park,19,19,19,19,19,19
East Tampa,3,3,3,3,3,3


In [54]:
# one hot encoding
tampa_onehot = pd.get_dummies(tampa_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
tampa_onehot['Neighborhood'] = tampa_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [tampa_onehot.columns[-1]] + list(tampa_onehot.columns[:-1])
tampa_onehot = tampa_onehot[fixed_columns]

tampa_onehot.head()

,Neighborhood,American Restaurant,Argentinian Restaurant,Art Gallery,Asian Restaurant,Athletics & Sports,BBQ Joint,Bagel Shop,Bakery,Bar,Beer Garden,Big Box Store,Bike Shop,Boat or Ferry,Bookstore,Bowling Alley,Breakfast Spot,Brewery,Bridal Shop,Burger Joint,Bus Station,Café,Chinese Restaurant,Cocktail Bar,Coffee Shop,Comic Shop,Construction & Landscaping,Convenience Store,Cuban Restaurant,Cycle Studio,Deli / Bodega,Discount Store,Donut Shop,Eastern European Restaurant,Electronics Store,Farm,Fast Food Restaurant,Fish Market,Flea Market,Flower Shop,Food,Food Court,Food Truck,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gastropub,Golf Course,Gym,Gym / Fitness Center,Health & Beauty Service,Home Service,Hotel,Ice Cream Shop,Insurance Office,Intersection,Japanese Restaurant,Jewelry Store,Juice Bar,Latin American Restaurant,Market,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Motorcycle Shop,Movie Theater,Music Venue,New American Restaurant,Other Nightlife,Other Repair Shop,Park,Pet Store,Pharmacy,Photography Studio,Pizza Place,Playground,Ramen Restaurant,Rental Car Location,Restaurant,Rock Club,Salon / Barbershop,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shipping Store,Shoe Store,Smoke Shop,Smoothie Shop,Soccer Field,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Supermarket,Supplement Shop,Sushi Restaurant,Tailor Shop,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Thrift / Vintage Store,Trail,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar
0,Harbour Island,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Harbour Island,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Harbour Island,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Harbour Island,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Harbour Island,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [55]:
tampa_onehot.shape

(219, 112)

In [56]:
# Let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category
tampa_grouped = tampa_onehot.groupby('Neighborhood').mean().reset_index()
tampa_grouped.head()

,Neighborhood,American Restaurant,Argentinian Restaurant,Art Gallery,Asian Restaurant,Athletics & Sports,BBQ Joint,Bagel Shop,Bakery,Bar,Beer Garden,Big Box Store,Bike Shop,Boat or Ferry,Bookstore,Bowling Alley,Breakfast Spot,Brewery,Bridal Shop,Burger Joint,Bus Station,Café,Chinese Restaurant,Cocktail Bar,Coffee Shop,Comic Shop,Construction & Landscaping,Convenience Store,Cuban Restaurant,Cycle Studio,Deli / Bodega,Discount Store,Donut Shop,Eastern European Restaurant,Electronics Store,Farm,Fast Food Restaurant,Fish Market,Flea Market,Flower Shop,Food,Food Court,Food Truck,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gastropub,Golf Course,Gym,Gym / Fitness Center,Health & Beauty Service,Home Service,Hotel,Ice Cream Shop,Insurance Office,Intersection,Japanese Restaurant,Jewelry Store,Juice Bar,Latin American Restaurant,Market,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Motorcycle Shop,Movie Theater,Music Venue,New American Restaurant,Other Nightlife,Other Repair Shop,Park,Pet Store,Pharmacy,Photography Studio,Pizza Place,Playground,Ramen Restaurant,Rental Car Location,Restaurant,Rock Club,Salon / Barbershop,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shipping Store,Shoe Store,Smoke Shop,Smoothie Shop,Soccer Field,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Supermarket,Supplement Shop,Sushi Restaurant,Tailor Shop,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Thrift / Vintage Store,Trail,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar
0,Audubon Park,0.050000,0.025,0.0,0.0,0.0,0.025,0.025,0.025,0.025,0.0,0.00,0.0,0.0,0.025,0.00,0.00,0.0,0.025,0.025,0.0,0.025,0.025,0.0,0.00,0.0,0.0,0.0,0.05,0.025,0.0,0.0,0.0,0.025,0.0,0.0,0.05,0.0,0.0,0.025,0.0,0.0,0.00,0.00,0.0,0.00,0.0,0.0,0.025,0.025,0.0,0.0,0.025,0.0,0.0,0.025,0.0,0.0,0.025000,0.0,0.0,0.0,0.025,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.000000,0.025,0.0,0.000000,0.025,0.0,0.025000,0.0,0.0,0.0,0.00,0.0,0.025,0.000000,0.00,0.025,0.05,0.0,0.0,0.00,0.0,0.0,0.05,0.0,0.0,0.0,0.025,0.0,0.0,0.025,0.0,0.025,0.0,0.0,0.05,0.0,0.0,0.0,0.00,0.025,0.0,0.0
1,Ballast Point,0.000000,0.000,0.0,0.0,0.0,0.000,0.000,0.000,0.000,0.0,0.04,0.0,0.0,0.000,0.04,0.04,0.0,0.000,0.000,0.0,0.000,0.040,0.0,0.04,0.0,0.0,0.0,0.04,0.000,0.0,0.0,0.0,0.000,0.0,0.0,0.04,0.0,0.0,0.000,0.0,0.0,0.04,0.04,0.0,0.04,0.0,0.0,0.000,0.000,0.0,0.0,0.000,0.0,0.0,0.000,0.0,0.0,0.000000,0.0,0.0,0.0,0.080,0.0,0.0,0.08,0.0,0.0,0.04,0.0,0.000000,0.000,0.0,0.000000,0.000,0.0,0.080000,0.0,0.0,0.0,0.04,0.0,0.040,0.040000,0.04,0.000,0.04,0.0,0.0,0.04,0.0,0.0,0.00,0.0,0.0,0.0,0.000,0.0,0.0,0.000,0.0,0.000,0.0,0.0,0.04,0.0,0.0,0.0,0.04,0.000,0.0,0.0
2,Bayshore Beautiful,0.166667,0.000,0.0,0.0,0.0,0.000,0.000,0.000,0.000,0.0,0.00,0.0,0.0,0.000,0.00,0.00,0.0,0.000,0.000,0.0,0.000,0.000,0.0,0.00,0.0,0.0,0.0,0.00,0.000,0.0,0.0,0.0,0.000,0.0,0.0,0.00,0.0,0.0,0.000,0.0,0.0,0.00,0.00,0.0,0.00,0.0,0.0,0.000,0.000,0.0,0.0,0.000,0.0,0.0,0.000,0.0,0.0,0.166667,0.0,0.0,0.0,0.000,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.166667,0.000,0.0,0.166667,0.000,0.0,0.166667,0.0,0.0,0.0,0.00,0.0,0.000,0.166667,0.00,0.000,0.00,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.000,0.0,0.0,0.000,0.0,0.000,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.000,0.0,0.0
3,Carver City-Lincoln Gardens,0.000000,0.000,0.0,0.0,0.0,0.000,0.000,0.000,0.000,0.0,0.00,0.0,0.0,0.000,0.00,0.10,0.0,0.000,0.000,0.0,0.000,0.000,0.0,0.00,0.0,0.0,0.0,0.00,0.000,0.0,0.1,0.0,0.000,0.0,0.0,0.00,0.0,0.0,0.100,0.1,0.0,0.00,0.00,0.1,0.00,0.0,0.0,0.000,0.000,0.0,0.0,0.000,0.0,0.0,0.000,0.0,0.0,0.000000,0.0,0.0,0.0,0.100,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.000000,0.000,0.0,0.000000,0.100,0.0,0.000000,0.0,0.0,0.0,0.00,0.0,0.000,0.000000,0.00,0.000,0.00,0.1,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.000,0.0,0.0,0.000,0.0,0.000,0.0,0.1,0.00,0.0,0.0,0.0,0.00,0.100,0.0,0.0
4,Citrus Park,0.000000,0.000,0.0,0.0,0.0,0.000,0.000,0.000,0.000,0.0,0.00,0.0,0.0,0.000,0.00,0.00,0.0,0.000,0.000,0.

In [57]:
# Let's confirm the new size
tampa_grouped.shape

(23, 112)

In [58]:
# Let's print each neighborhood along with the top 5 most common venues
num_top_venues = 5

for hood in tampa_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = tampa_grouped[tampa_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Audubon Park----
                  venue  freq
0   American Restaurant  0.05
1  Fast Food Restaurant  0.05
2      Cuban Restaurant  0.05
3       Thai Restaurant  0.05
4                   Spa  0.05


----Ballast Point----
                    venue  freq
0      Mexican Restaurant  0.08
1       Mobile Phone Shop  0.08
2             Pizza Place  0.08
3     Fried Chicken Joint  0.04
4  Furniture / Home Store  0.04


----Bayshore Beautiful----
                 venue  freq
0  American Restaurant  0.17
1       Sandwich Place  0.17
2      Other Nightlife  0.17
3            Pet Store  0.17
4          Pizza Place  0.17


----Carver City-Lincoln Gardens----
            venue  freq
0  Breakfast Spot   0.1
1        Tea Room   0.1
2      Shoe Store   0.1
3     Flower Shop   0.1
4            Food   0.1


----Citrus Park----
                        venue  freq
0                   Rock Club   0.2
1  Construction & Landscaping   0.2
2                        Farm   0.2
3                Home Service   

In [69]:
# Let's put that into a pandas dataframe. 
# First, let's write a function to sort the venues in descending order

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [70]:
# Now let's create the new dataframe and display the top 25 venues for each neighborhood
num_top_venues = 25

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = tampa_grouped['Neighborhood']

for ind in np.arange(tampa_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(tampa_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue,21th Most Common Venue,22th Most Common Venue,23th Most Common Venue,24th Most Common Venue,25th Most Common Venue
0,Audubon Park,American Restaurant,Thai Restaurant,Spa,Cuban Restaurant,Fast Food Restaurant,Shipping Store,Flower Shop,Pizza Place,Seafood Restaurant,Burger Joint,Gym / Fitness Center,Salon / Barbershop,Café,Chinese Restaurant,Pharmacy,Juice Bar,Hotel,Other Repair Shop,Bridal Shop,Eastern European Restaurant,Intersection,Mexican Restaurant,Cycle Studio,Gym,Sports Bar
1,Ballast Point,Mexican Restaurant,Pizza Place,Mobile Phone Shop,Fast Food Restaurant,Coffee Shop,Restaurant,Salon / Barbershop,Sandwich Place,Scenic Lookout,Chinese Restaurant,Shipping Store,Music Venue,Smoothie Shop,Breakfast Spot,Bowling Alley,Cuban Restaurant,Thai Restaurant,Furniture / Home Store,Big Box Store,Video Game Store,Food Truck,Fried Chicken Joint,Deli / Bodega,Golf Course,Discount Store
2,Bayshore Beautiful,American Restaurant,Juice Bar,Pizza Place,Pet Store,Other Nightlife,Sandwich Place,Gym,Fish Market,Deli / Bodega,Discount Store,Donut Shop,Eastern European Restaurant,Home Service,Electronics Store,Farm,Health & Beauty Service,Fast Food Restaurant,Gym / Fitness Center,Golf Course,Flea Market,Flower Shop,Cycle Studio,Food Court,Food Truck,Fried Chicken Joint
3,Carver City-Lincoln Gardens,Shoe Store,Pharmacy,Flower Shop,Food,Fruit & Vegetable Store,Mexican Restaurant,Tea Room,Breakfast Spot,Discount Store,Video Store,Eastern European Restaurant,Flea Market,Electronics Store,Donut Shop,Farm,Fast Food Restaurant,Fish Market,Deli / Bodega,Wine Bar,Food Court,Cuban Restaurant,Food Truck,Fried Chicken Joint,Furniture / Home Store,Gastropub
4,Citrus Park,Rock Club,Scenic Lookout,Home Service,Construction & Landscaping,Farm,Flower Shop,Cycle Studio,Deli / Bodega,Discount Store,Donut Shop,Eastern European Restaurant,Electronics Store,Fast Food Restaurant,Fish Market,Flea Market,Wine Bar,Cuban Restaurant,Food Court,Food Truck,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gastropub,Golf Course,Gym


In [71]:
neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue,21th Most Common Venue,22th Most Common Venue,23th Most Common Venue,24th Most Common Venue,25th Most Common Venue
0,Audubon Park,American Restaurant,Thai Restaurant,Spa,Cuban Restaurant,Fast Food Restaurant,Shipping Store,Flower Shop,Pizza Place,Seafood Restaurant,Burger Joint,Gym / Fitness Center,Salon / Barbershop,Café,Chinese Restaurant,Pharmacy,Juice Bar,Hotel,Other Repair Shop,Bridal Shop,Eastern European Restaurant,Intersection,Mexican Restaurant,Cycle Studio,Gym,Sports Bar
1,Ballast Point,Mexican Restaurant,Pizza Place,Mobile Phone Shop,Fast Food Restaurant,Coffee Shop,Restaurant,Salon / Barbershop,Sandwich Place,Scenic Lookout,Chinese Restaurant,Shipping Store,Music Venue,Smoothie Shop,Breakfast Spot,Bowling Alley,Cuban Restaurant,Thai Restaurant,Furniture / Home Store,Big Box Store,Video Game Store,Food Truck,Fried Chicken Joint,Deli / Bodega,Golf Course,Discount Store
2,Bayshore Beautiful,American Restaurant,Juice Bar,Pizza Place,Pet Store,Other Nightlife,Sandwich Place,Gym,Fish Market,Deli / Bodega,Discount Store,Donut Shop,Eastern European Restaurant,Home Service,Electronics Store,Farm,Health & Beauty Service,Fast Food Restaurant,Gym / Fitness Center,Golf Course,Flea Market,Flower Shop,Cycle Studio,Food Court,Food Truck,Fried Chicken Joint
3,Carver City-Lincoln Gardens,Shoe Store,Pharmacy,Flower Shop,Food,Fruit & Vegetable Store,Mexican Restaurant,Tea Room,Breakfast Spot,Discount Store,Video Store,Eastern European Restaurant,Flea Market,Electronics Store,Donut Shop,Farm,Fast Food Restaurant,Fish Market,Deli / Bodega,Wine Bar,Food Court,Cuban Restaurant,Food Truck,Fried Chicken Joint,Furniture / Home Store,Gastropub
4,Citrus Park,Rock Club,Scenic Lookout,Home Service,Construction & Landscaping,Farm,Flower Shop,Cycle Studio,Deli / Bodega,Discount Store,Donut Shop,Eastern European Restaurant,Electronics Store,Fast Food Restaurant,Fish Market,Flea Market,Wine Bar,Cuban Restaurant,Food Court,Food Truck,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gastropub,Golf Course,Gym
5,College Hill,Construction & Landscaping,Trail,BBQ Joint,Electronics Store,Wine Bar,Food Court,Discount Store,Donut Shop,Eastern European Restaurant,Farm,Fast Food Restaurant,Fish Market,Flea Market,Flower Shop,Food,Fried Chicken Joint,Food Truck,Cycle Studio,Fruit & Vegetable Store,Furniture / Home Store,Gastropub,Golf Course,Gym,Gym / Fitness Center,Health & Beauty Service
6,Dixie Farms,American Restaurant,Comic Shop,Playground,Photography Studio,Miscellaneous Shop,Food,Discount Store,Donut Shop,Eastern European Restaurant,Electronics Store,Farm,Fast Food Restaurant,Fish Market,Flea Market,Flower Shop,Food Court,Cycle Studio,Food Truck,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gastropub,Golf Course,Gym,Gym / Fitness Center
7,Drew Park,Furniture / Home Store,Pizza Place,Donut Shop,Ice Cream Shop,Food Truck,Deli / Bodega,Pharmacy,Smoke Shop,Fast Food Restaurant,Spanish Restaurant,Fish Market,Motorcycle Shop,Rental Car Location,Video Store,Gym / Fitness Center,Eastern European Restaurant,Cycle Studio,Health & Beauty Service,Discount Store,Gym,Golf Course,Food Court,Food,Farm,Gastropub
8,East Tampa,Sandwich Place,Pharmacy,Fast Food Restaurant,Insurance Office,Cycle Studio,Deli / Bodega,Discount Store,Donut Shop,Eastern European Restaurant,Electronics Store,Farm,Fish Market,Flea Market,Flower Shop,Food,Food Court,Food Truck,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gastropub,Golf Course,Gym,Gym / Fitness Center,Health & Beauty

In [72]:
# set number of clusters
kclusters = 5

tampa_grouped_clustering = tampa_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(tampa_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int32)

In [73]:
tampa_data.head()

,PostalCode,Neighborhood,Latitude,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue,21th Most Common Venue,22th Most Common Venue,23th Most Common Venue,24th Most Common Venue,25th Most Common Venue
0,33602,Harbour Island,27.961,-82.460,Coffee Shop,New American Restaurant,Wine Bar,Steakhouse,Ice Cream Shop,Latin American Restaurant,Market,Food Court,Park,Ramen Restaurant,Cuban Restaurant,Cocktail Bar,Café,Brewery,Gym / Fitness Center,Sushi Restaurant,Supermarket,Thrift / Vintage Store,Big Box Store,Farm,Athletics & Sports,Fast Food Restaurant,Fish Market,Flea Market,Flower Shop
1,33603,Lowry Park Central,27.985,-82.463,Intersection,Brewery,Japanese Restaurant,Bike Shop,Pharmacy,Coffee Shop,Burger Joint,Southern / Soul Food Restaurant,Sporting Goods Shop,Playground,Insurance Office,Art Gallery,Jewelry Store,Flower Shop,Donut Shop,Eastern European Restaurant,Electronics Store,Farm,Fast Food Restaurant,Fish Market,Flea Market,Golf Course,Gastropub,Food,Gym / Fitness Center
2,33604,River Grove,28.017,-82.458,Ice Cream Shop,Speakeasy,Scenic Lookout,Pizza Place,Food,Gym / Fitness Center,Gym,Hotel,Deli / Bodega,Discount Store,Donut Shop,Home Service,Eastern European Restaurant,Electronics Store,Farm,Fast Food Restaurant,Fish Market,Health & Beauty Service,Flea Market,Flower Shop,Cuban Restaurant,Food Court,Food Truck,Fried Chicken Joint,Fruit & Vegetable Store
3,33605,East Tampa,27.967,-82.433,Sandwich Place,Pharmacy,Fast Food Restaurant,Insurance Office,Cycle Studio,Deli / Bodega,Discount Store,Donut Shop,Eastern European Restaurant,Electronics Store,Farm,Fish Market,Flea Market,Flower Shop,Food,Food Court,Food Truck,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gastropub,Golf Course,Gym,Gym / Fitness Center,Health & Beauty Service
4,33606,Historic Hyde Park North,27.948,-82.458,Coffee Shop,Pizza Place,Bar,American Restaurant,Bookstore,Café,Scenic Lookout,Cuban Restaurant,Park,Movie Theater,Flea Market,Mediterranean Restaurant,Market,Vietnamese Restaurant,Ice Cream Shop,Hotel,Health & Beauty Service,Breakfast Spot,Restaurant,Spanish Restaurant,Sports Bar,Asian Restaurant,Bagel Shop,Tapas Restaurant,Beer Garden


In [74]:
# Let's take the columns we need
tampa_data = tampa_data[['PostalCode','Neighborhood','Latitude','Longitude']].reset_index(drop=True)
tampa_data.head()

,PostalCode,Neighborhood,Latitude,Longitude
0,33602,Harbour Island,27.961,-82.460
1,33603,Lowry Park Central,27.985,-82.463
2,33604,River Grove,28.017,-82.458
3,33605,East Tampa,27.967,-82.433
4,33606,Historic Hyde Park North,27.948,-82.458


In [75]:
# Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
# merge dataframes to add latitude/longitude for each neighborhood
tampa_data = tampa_data.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

tampa_data.head() # check the last columns!

,PostalCode,Neighborhood,Latitude,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue,21th Most Common Venue,22th Most Common Venue,23th Most Common Venue,24th Most Common Venue,25th Most Common Venue
0,33602,Harbour Island,27.961,-82.460,Coffee Shop,New American Restaurant,Wine Bar,Steakhouse,Ice Cream Shop,Latin American Restaurant,Market,Food Court,Park,Ramen Restaurant,Cuban Restaurant,Cocktail Bar,Café,Brewery,Gym / Fitness Center,Sushi Restaurant,Supermarket,Thrift / Vintage Store,Big Box Store,Farm,Athletics & Sports,Fast Food Restaurant,Fish Market,Flea Market,Flower Shop
1,33603,Lowry Park Central,27.985,-82.463,Intersection,Brewery,Japanese Restaurant,Bike Shop,Pharmacy,Coffee Shop,Burger Joint,Southern / Soul Food Restaurant,Sporting Goods Shop,Playground,Insurance Office,Art Gallery,Jewelry Store,Flower Shop,Donut Shop,Eastern European Restaurant,Electronics Store,Farm,Fast Food Restaurant,Fish Market,Flea Market,Golf Course,Gastropub,Food,Gym / Fitness Center
2,33604,River Grove,28.017,-82.458,Ice Cream Shop,Speakeasy,Scenic Lookout,Pizza Place,Food,Gym / Fitness Center,Gym,Hotel,Deli / Bodega,Discount Store,Donut Shop,Home Service,Eastern European Restaurant,Electronics Store,Farm,Fast Food Restaurant,Fish Market,Health & Beauty Service,Flea Market,Flower Shop,Cuban Restaurant,Food Court,Food Truck,Fried Chicken Joint,Fruit & Vegetable Store
3,33605,East Tampa,27.967,-82.433,Sandwich Place,Pharmacy,Fast Food Restaurant,Insurance Office,Cycle Studio,Deli / Bodega,Discount Store,Donut Shop,Eastern European Restaurant,Electronics Store,Farm,Fish Market,Flea Market,Flower Shop,Food,Food Court,Food Truck,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gastropub,Golf Course,Gym,Gym / Fitness Center,Health & Beauty Service
4,33606,Historic Hyde Park North,27.948,-82.458,Coffee Shop,Pizza Place,Bar,American Restaurant,Bookstore,Café,Scenic Lookout,Cuban Restaurant,Park,Movie Theater,Flea Market,Mediterranean Restaurant,Market,Vietnamese Restaurant,Ice Cream Shop,Hotel,Health & Beauty Service,Breakfast Spot,Restaurant,Spanish Restaurant,Sports Bar,Asian Restaurant,Bagel Shop,Tapas Restaurant,Beer Garden


In [76]:
# There are NaN values in column Cluster Labels - let's take only the rows where Cluster label is not NaN
tampa_data = tampa_data[tampa_data['Cluster Labels'].notna()]
tampa_data.head()

KeyError: 'Cluster Labels'

In [77]:

# Lets change the column 'Cluster Labels' to integer
tampa_data['Cluster Labels'] = tampa_data['Cluster Labels'].astype('int')

KeyError: 'Cluster Labels'

In [68]:
# Finally, let's visualize the resulting clusters

map_clusters = folium.Map(location=[latitude, longitude], zoom_start=10)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(tampa_data['Latitude'], tampa_data['Longitude'], tampa_data['Neighborhood'], tampa_data['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

KeyError: 'Cluster Labels'

In [60]:
# Dataframe of top10 categories
neighborhoods_top10 = neighborhoods_venues_sorted[['Neighborhood',
 '1st Most Common Venue',
 '2nd Most Common Venue',
 '3rd Most Common Venue',
 '4th Most Common Venue',
 '5th Most Common Venue',
 '6th Most Common Venue',
 '7th Most Common Venue',
 '8th Most Common Venue',
 '9th Most Common Venue',
 '10th Most Common Venue']].set_index('Neighborhood')
neighborhoods_top10

,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
Neighborhood,,,,,,,,,,
Audubon Park,Thai Restaurant,Shipping Store,Cuban Restaurant,American Restaurant,Other Repair Shop,Salon / Barbershop,Burger Joint,Café,Gym,Clothing Store
Ballast Point,Mexican Restaurant,Pizza Place,Mobile Phone Shop,Restaurant,Cosmetics Shop,Coffee Shop,Chinese Restaurant,Salon / Barbershop,Sandwich Place,Scenic Lookout
Bayshore Beautiful,American Restaurant,Sandwich Place,Juice Bar,Pizza Place,Other Nightlife,Food,Deli / Bodega,Discount Store,Donut Shop,Farm
Carver City-Lincoln Gardens,Tea Room,Pharmacy,Breakfast Spot,Fruit & Vegetable Store,Mexican Restaurant,Shoe Store,Flower Shop,Discount Store,Video Store,Farm
Citrus Park,Scenic Lookout,Farm,Rock Club,Wine Bar,Food Court,Cycle Studio,Deli / Bodega,Discount Store,Donut Shop,Fast Food Restaurant
College Hill,Trail,Wine Bar,Fried Chicken Joint,Deli / Bodega,Discount Store,Donut Shop,Farm,Fast Food Restaurant,Fish Market,Flea Market
Dixie Farms,Video Store,Photography Studio,Business Service,Miscellaneous Shop,Wine Bar,Food Truck,Discount Store,Donut Shop,Farm,Fast Food Restaurant
Drew Park,Donut Shop,Pizza Place,Fast Food Restaurant,Ice Cream Shop,Furniture / Home Store,Deli / Bodega,Pharmacy,Fish Market,Motorcycle Shop,Food Truck
East Tampa,Sandwich Place,Fast Food Restaurant,Pharmacy,Food Court,Cycle Studio,Deli / Bodega,Discount Store,Donut Shop,Farm,Fish Market



Examine the Clusters

In [63]:
# Cluster 0 - Red
tampa_data.loc[tampa_data['Cluster Labels'] == 0, tampa_data.columns[[2] + list(range(5, tampa_data.shape[1]))]]

,Latitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue,21th Most Common Venue,22th Most Common Venue,23th Most Common Venue,24th Most Common Venue,25th Most Common Venue
0,27.961,Coffee Shop,Park,New American Restaurant,Wine Bar,Market,Ramen Restaurant,Brewery,Food Court,Steakhouse,Supermarket,Cuban Restaurant,Sushi Restaurant,Thrift / Vintage Store,Latin American Restaurant,Ice Cream Shop,Gym / Fitness Center,Deli / Bodega,Discount Store,Donut Shop,Gym,Farm,Fast Food Restaurant,Golf Course,Furniture / Home Store,Fish Market
1,27.985,Intersection,Brewery,Japanese Restaurant,Bike Shop,Coffee Shop,Burger Joint,Southern / Soul Food Restaurant,Sporting Goods Shop,Playground,Lawyer,Auto Garage,Art Gallery,Gym / Fitness Center,Food Court,Donut Shop,Farm,Fast Food Restaurant,Fish Market,Flea Market,Flower Shop,Food,Health & Beauty Service,Home Service,Gym,Food Truck
2,28.017,Ice Cream Shop,Skating Rink,Pizza Place,Food,Food Court,Deli / Bodega,Discount Store,Donut Shop,Farm,Fast Food Restaurant,Fish Market,Flea Market,Flower Shop,Food Truck,Cuban Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gastropub,Golf Course,Gym,Gym / Fitness Center,Gym Pool,Health & Beauty Service,Home Service
3,27.967,Sandwich Place,Fast Food Restaurant,Pharmacy,Food Court,Cycle Studio,Deli / Bodega,Discount Store,Donut Shop,Farm,Fish Market,Flea Market,Flower Shop,Food,Wine Bar,Cuban Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gastropub,Golf Course,Gym,Gym / Fitness Center,Gym Pool,Health & Beauty Service,Home Service
4,27.948,Coffee Shop,Bar,Pizza Place,Hotel,Asian Restaurant,Sports Bar,Café,Restaurant,Park,Movie Theater,Mediterranean Restaurant,Flea Market,Breakfast Spot,Market,Vietnamese Restaurant,Gastropub,Ice Cream Shop,Health & Beauty Service,Scenic Lookout,American Restaurant,Bookstore,Bagel Shop,Art Museum,Steakhouse,Speakeasy
5,27.963,Tea Room,Pharmacy,Breakfast Spot,Fruit & Vegetable Store,Mexican Restaurant,Shoe Store,Flower Shop,Discount Store,Video Store,Farm,Food,Fast Food Restaurant,Donut Shop,Fish Market,Deli / Bodega,Flea Market,Wine Bar,Food Truck,Food Court,Cuban Restaurant,Fried Chicken Joint,Furniture / Home Store,Gastropub,Golf Course,Gym
6,27.942,Thai Restaurant,Shipping Store,Cuban Restaurant,American Restaurant,Other Repair Shop,Salon / Barbershop,Burger Joint,Café,Gym,Clothing Store,Pizza Place,Pharmacy,Cycle Studio,Fast Food Restaurant,Mexican Restaurant,Flower Shop,Juice Bar,Intersection,Hotel,Furniture / Home Store,Seafood Restaurant,Gym / Fitness Center,Sports Bar,Bar,Video Store
7,27.995,Hotel,Discount Store,Furniture / Home Store,Food,Wine Bar,Food Truck,Deli / Bodega,Donut Shop,Farm,Fast Food Restaurant,Fish Market,Flea Market,Flower Shop,Food Court,Fried Chicken Joint,Cuban Restaurant,Fruit & Vegetable Store,Gastropub,Golf Course,Gym,Gym / Fitness Center,Gym Pool,Health & Beauty Service,Home Service,Ice Cream Shop
8,27.891,Mexican Restaurant,Pizza Place,Mobile Phone Shop,Restaurant,Cosmetics Shop,Coffee Shop,Chinese Restaurant,Salon / Barbershop,Sandwich Place,Scenic Lookout,Fast Food Restaurant,Shipping Store,Smoothie Shop,Breakfast Spot,Bowling Alley,Big Box Store,Food Truck,Video Game Store,Furniture / Home Store,Fried Chicken Joint,Thai Restaurant,Cuban Restaurant,Food Court,Fish Market,Food
10,28.077,Sports Bar,Nature Preserve,Pizza Place,Golf Course,Mexican Restaurant,Bar,Middle Eastern Restaurant,Discount Store,Donut Shop,Farm,Fast Food Restaurant,Fish Market,Flea Market,Flower Shop,Food,Food Court,Wine Bar,Food Truck,Fried Chicken Joint,Cycle Studio,Frui

In [64]:

# Cluster 1 - Purple
tampa_data.loc[tampa_data['Cluster Labels'] == 1, tampa_data.columns[[2] + list(range(5, tampa_data.shape[1]))]]

,Latitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue,21th Most Common Venue,22th Most Common Venue,23th Most Common Venue,24th Most Common Venue,25th Most Common Venue
20,28.007,Sandwich Place,Food Truck,Cycle Studio,Deli / Bodega,Discount Store,Donut Shop,Farm,Fast Food Restaurant,Fish Market,Flea Market,Flower Shop,Food,Food Court,Wine Bar,Cuban Restaurant,Fruit & Vegetable Store,Furniture / Home Store,Gastropub,Golf Course,Gym,Gym / Fitness Center,Gym Pool,Health & Beauty Service,Home Service,Hotel


In [97]:
# Cluster 2 - Light Blue
tampa_data.loc[tampa_data['Cluster Labels'] == 2, tampa_data.columns[[2] + list(range(5, tampa_data.shape[1]))]

SyntaxError: unexpected EOF while parsing (<ipython-input-97-030eeeaf59cb>, line 2)

In [69]:
# Cluster 4 - Orange
tampa_data.loc[tampa_data['Cluster Labels'] == 4, tampa_data.columns[[2] + list(range(5, s.shape[1]))]]

,Latitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue,21th Most Common Venue,22th Most Common Venue,23th Most Common Venue,24th Most Common Venue,25th Most Common Venue
9,28.05,Convenience Store,Indian Restaurant,Food Truck,Deli / Bodega,Discount Store,Donut Shop,Farm,Fast Food Restaurant,Fish Market,Flea Market,Flower Shop,Food,Food Court,Fried Chicken Joint,Cuban Restaurant,Fruit & Vegetable Store,Furniture / Home Store,Gastropub,Golf Course,Gym,Gym / Fitness Center,Gym Pool,Health & Beauty Service,Home Service,Hotel


## Other Fish Market and Venues

In [82]:
# Other pet stores
fish_market = tampa_venues.loc[tampa_venues['Venue Category'] ==  'Fish Market']
fish_market

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
156,Drew Park,28.009,-82.503,Cox's Seafood Market,28.009831,-82.504553,Fish Market


In [83]:

fish_market_g = pd.DataFrame(fish_market.groupby('Neighborhood')['Venue'].nunique())
fish_market_g.rename(columns = {'Venue':'Fish_market'}, inplace=True)
fish_market_g

,Fish_market
Neighborhood,
Drew Park,1


In [85]:
# Other pet related business?
market_venues_list = ['Supermarket', 'Seafood Restairamt', 'Food Court', 'Veterinarian', 'Pet Service']
market_venues = tampa_venues[tampa_venues['Venue Category'].str.contains('|'.join(market_venues_list))]
market_venues

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
6,Harbour Island,27.961,-82.46,The Hall On Franklin,27.959731,-82.460077,Food Court
11,Harbour Island,27.961,-82.46,Oceanic Supermarket,27.957732,-82.460625,Supermarket


In [91]:

# What about parks?
Restaurant = tampa_venues.loc[tampa_venues['Venue Category'] ==  'Restaurant']
Restaurant

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
44,Historic Hyde Park North,27.948,-82.458,SoFresh,27.949004,-82.458581,Restaurant
136,Ballast Point,27.891,-82.507,Casa Mexicana Bar & Grill,27.893086,-82.504068,Restaurant


In [109]:
restaurant_g = pd.DataFrame(Restaurant.groupby('Neighborhood')['Venue'].nunique())
restaurant_g.rename(columns = {'Venue':'Restaurant'}, inplace=True)
restaurant_g

,Restaurant
Neighborhood,
Ballast Point,1
Historic Hyde Park North,1


In [112]:
# Get the data
df_populdata = pd.read_csv('tampazipcode.csv')
df_populdata.head()

,Code,Country,Admin1,Admin2,Admin3,Admin4,Population,Households per ZIP Code,Average House Value,Average Income Per Household,Male Population,Female Population,Male Median Age,Female Median Age,Unemployment Rate,Unnamed: 15
0,33602,United States,Florida,Hillsborough,27.961,-82.460,17412,6004,272000,67945,6092,5423,35.8,34.5,12.1%,NaN
1,33603,United States,Florida,Hillsborough,27.985,-82.463,20303,7721,147600,40078,9421,9679,35.4,37.5,12.1%,NaN
2,33604,United States,Florida,Hillsborough,28.017,-82.458,38971,13944,118400,37316,17443,18042,35.6,37.0,12.1%,NaN
3,33605,United States,Florida,Hillsborough,27.967,-82.433,18689,6481,84500,26121,8469,8604,34.6,35.8,12.1%,NaN
4,33606,United States,Florida,Hillsborough,27.948,-82.458,18271,7517,452900,74855,8632,9114,32.1,29.4,12.1%,NaN


In [119]:
# Let's take the columns we need
df_populdata = df_populdata[['Code','Households per ZIP Code']].reset_index(drop=True)
df_populdata.head()

,Code,Households per ZIP Code
0,33602,6004
1,33603,7721
2,33604,13944
3,33605,6481
4,33606,7517


In [121]:
# Rename the columns
df_populdata.rename(columns = {'Code': 'PostalCode',}, inplace=True)
df_populdata.head()

,PostalCode,Households per ZIP Code
0,33602,6004
1,33603,7721
2,33604,13944
3,33605,6481
4,33606,7517


In [134]:
# Merge the dataframes
tampa_dataall = pd.merge(tampa_data, df_populdata, on='PostalCode', how='inner')
tampa_dataall

,PostalCode,Neighborhood,Latitude,Longitude,Households per ZIP Code
0,33602,Harbour Island,27.961,-82.460,6004
1,33603,Lowry Park Central,27.985,-82.463,7721
2,33604,River Grove,28.017,-82.458,13944
3,33605,East Tampa,27.967,-82.433,6481
4,33606,Historic Hyde Park North,27.948,-82.458,7517
5,33607,Carver City-Lincoln Gardens,27.963,-82.490,9552
6,33609,Audubon Park,27.942,-82.506,766
7,33610,Eastern Heights,27.995,-82.405,14245
8,33611,Ballast Point,27.891,-82.507,14148
9,33612,Forest Hills,28.050,-82.450,17612


In [138]:
# Merge the dataframes
df_tampa_all = tampa_dataall.reset_index()
df_tampa_all = pd.merge(pd.merge(pd.merge(tampa_dataall, restaurant_g, on='Neighborhood', how='left'), 
        fish_market, on='Neighborhood', how='left'), market_venues, on='Neighborhood', how='left')
df_tampa_all.fillna('-', inplace=True)
df_tampa_all

,PostalCode,Neighborhood,Latitude,Longitude,Households per ZIP Code,Restaurant,Neighborhood Latitude_x,Neighborhood Longitude_x,Venue_x,Venue Latitude_x,Venue Longitude_x,Venue Category_x,Neighborhood Latitude_y,Neighborhood Longitude_y,Venue_y,Venue Latitude_y,Venue Longitude_y,Venue Category_y
0,33602,Harbour Island,27.961,-82.460,6004,-,-,-,-,-,-,-,27.961,-82.46,The Hall On Franklin,27.9597,-82.4601,Food Court
1,33602,Harbour Island,27.961,-82.460,6004,-,-,-,-,-,-,-,27.961,-82.46,Oceanic Supermarket,27.9577,-82.4606,Supermarket
2,33603,Lowry Park Central,27.985,-82.463,7721,-,-,-,-,-,-,-,-,-,-,-,-,-
3,33604,River Grove,28.017,-82.458,13944,-,-,-,-,-,-,-,-,-,-,-,-,-
4,33605,East Tampa,27.967,-82.433,6481,-,-,-,-,-,-,-,-,-,-,-,-,-
5,33606,Historic Hyde Park North,27.948,-82.458,7517,1,-,-,-,-,-,-,-,-,-,-,-,-
6,33607,Carver City-Lincoln Gardens,27.963,-82.490,9552,-,-,-,-,-,-,-,-,-,-,-,-,-
7,33609,Audubon Park,27.942,-82.506,766,-,-,-,-,-,-,-,-,-,-,-,-,-
8,33610,Eastern Heights,27.995,-82.405,14245,-,-,-,-,-,-,-,-,-,-,-,-,-
9,33611,Ballast Point,27.891,-82.507,14148,1,-,-,-,-,-,-,-,-,-,-,-,-
